# SharePoint to Fabric Lakehouse ETL

## Overview
This notebook automates the process of downloading Excel files from SharePoint and loading them into Microsoft Fabric Lakehouse.

### What it does:
- Downloads Excel files from SharePoint using sharing links
- Writes files to Fabric Lakehouse
- Overwrites existing files on each run
- Provides detailed logging and error reporting

### Prerequisites:
- Microsoft Fabric workspace with a Lakehouse
- SharePoint files with "Anyone with link can edit" sharing enabled
- Proper permissions to write to the Lakehouse

---


## 🚀 Setup Instructions

### Step 1: Get SharePoint Sharing Links
For each Excel file you want to sync:
1. Navigate to the file in SharePoint
2. Right-click the file → **Share**
3. Select **"Anyone with the link can edit"**
4. Click **Copy link**
5. Save this link - you'll need it in the configuration below

### Step 2: Get Your Lakehouse Path
1. In Fabric, open your Lakehouse
2. Click on the **Files** folder
3. Copy the ABFS path (format: `abfss://workspace@onelake.dfs.fabric.microsoft.com/lakehouse.Lakehouse/Files/folder`)

### Step 3: Configure the notebook
Update the configuration in **Cell 4** below with your values.

---

In [ ]:
# Install required libraries
!pip install requests --quiet
print("✓ Dependencies installed")

In [ ]:
from pyspark.sql import SparkSession
import requests
from io import BytesIO
from datetime import datetime
from notebookutils import mssparkutils
from urllib.parse import unquote
import os

## ⚙️ Configuration Section

**⚠️ IMPORTANT: Update all values below before running the notebook**

Replace the placeholder values with your actual:
- Lakehouse ABFS path
- SharePoint tenant and site names
- File URLs and sharing links

---

In [ ]:
# ============================================================================
# CONFIGURATION - UPDATE THESE VALUES
# ============================================================================

# Lakehouse Configuration
# Format: abfss://WORKSPACE_NAME@onelake.dfs.fabric.microsoft.com/LAKEHOUSE_NAME.Lakehouse/Files/FOLDER_PATH
lakehouse_abfs_path = "abfss://YOUR_WORKSPACE@onelake.dfs.fabric.microsoft.com/YOUR_LAKEHOUSE.Lakehouse/Files/YOUR_FOLDER"

# SharePoint Configuration
sharepoint_tenant = "your-tenant-name"  # e.g., "contoso"
sharepoint_site = "your-site-name"      # e.g., "finance" or "projects"

# Source Files Configuration
# For each file, provide:
# 1. Original SharePoint URL (for reference)
# 2. Sharing link (Right-click file > Share > "Anyone with link can edit" > Copy link)
# 3. Desired filename in lakehouse
# 4. Description (for logging purposes)

source_files = [
    {
        "url": "https://YOUR_TENANT.sharepoint.com/sites/YOUR_SITE/Shared%20Documents/Path/To/File1.xlsx",
        "sharing_link": "",  # PASTE YOUR SHARING LINK HERE
        "lakehouse_name": "File1.xlsx",
        "description": "First Excel File"
    },
    {
        "url": "https://YOUR_TENANT.sharepoint.com/sites/YOUR_SITE/Shared%20Documents/Path/To/File2.xlsx",
        "sharing_link": "",  # PASTE YOUR SHARING LINK HERE
        "lakehouse_name": "File2.xlsx",
        "description": "Second Excel File"
    }
    # Add more files as needed following the same pattern
]

# ============================================================================
# END CONFIGURATION
# ============================================================================

print("✓ Configuration loaded")
print(f"  Target Lakehouse: {lakehouse_abfs_path}")
print(f"  Files to process: {len(source_files)}")
print("\nSharing links status:")
for file in source_files:
    status = "✓" if file.get('sharing_link') else "✗ MISSING"
    print(f"  {status} {file['lakehouse_name']}")

## 📦 Core Functions

The following cells define the ETL functions. **No changes needed in this section.**

### Functions:
1. **download_from_sharepoint()** - Downloads files using sharing links
2. **write_to_lakehouse()** - Writes files to Lakehouse with overwrite
3. **run_etl()** - Main orchestration function

---

In [ ]:
def download_from_sharepoint(file_config):
    """Download Excel file from SharePoint using sharing link"""
    try:
        sharing_link = file_config.get('sharing_link', '')
        
        if not sharing_link:
            print("  ✗ No sharing link configured")
            return None
        
        # Convert sharing link to direct download URL
        if 'sharepoint.com' in sharing_link:
            base_url = sharing_link.split('?')[0]
            download_url = base_url + '?download=1'
        else:
            download_url = sharing_link
        
        print("  Downloading from SharePoint...")
        response = requests.get(download_url, allow_redirects=True, timeout=60)
        
        # Verify successful download
        if response.status_code == 200 and len(response.content) > 5000:
            # Check if content is Excel file (ZIP format)
            if response.content[:2] == b'PK':
                print(f"  ✓ Downloaded successfully ({len(response.content):,} bytes)")
                return BytesIO(response.content)
            else:
                print("  ✗ Content is not an Excel file")
        else:
            print(f"  ✗ Download failed: Status {response.status_code}")
        
        return None
            
    except Exception as e:
        print(f"  ✗ Error: {str(e)}")
        return None

In [ ]:
def write_to_lakehouse(file_bytes, lakehouse_path, filename):
    """Write file to Fabric Lakehouse (overwrites if exists)"""
    try:
        full_path = f"{lakehouse_path}/{filename}"
        file_data = file_bytes.getvalue()
        
        # Write to temporary local file
        temp_path = f"/tmp/{filename}"
        with open(temp_path, 'wb') as f:
            f.write(file_data)
        
        print(f"  ✓ Temporary file created ({len(file_data):,} bytes)")
        
        # Remove existing file if it exists
        try:
            mssparkutils.fs.rm(full_path)
            print("  ✓ Existing file removed")
        except:
            print("  No existing file to remove")
        
        # Copy to lakehouse
        print("  Writing to lakehouse...")
        mssparkutils.fs.cp(f"file://{temp_path}", full_path)
        
        # Clean up temporary file
        os.remove(temp_path)
        
        print(f"  ✓ Successfully written to lakehouse")
        return True
        
    except Exception as e:
        print(f"  ✗ Error writing to lakehouse: {str(e)}")
        return False

In [ ]:
def run_etl():
    """Main ETL process - Downloads files from SharePoint and writes to Lakehouse"""
    print("=" * 80)
    print(f"Starting ETL Process - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)
    
    success_count = 0
    failed_count = 0
    results = []
    
    for idx, file_config in enumerate(source_files, 1):
        print(f"\n[{idx}/{len(source_files)}] Processing: {file_config['description']}")
        print("-" * 80)
        print(f"Target: {file_config['lakehouse_name']}")
        print()
        
        # Download from SharePoint
        file_bytes = download_from_sharepoint(file_config)
        
        if file_bytes:
            # Write to Lakehouse
            success = write_to_lakehouse(
                file_bytes,
                lakehouse_abfs_path,
                file_config['lakehouse_name']
            )
            
            if success:
                success_count += 1
                results.append({
                    'file': file_config['lakehouse_name'],
                    'status': 'SUCCESS',
                    'timestamp': datetime.now().isoformat()
                })
                print(f"\n✓ {file_config['lakehouse_name']} - COMPLETED SUCCESSFULLY")
            else:
                failed_count += 1
                results.append({
                    'file': file_config['lakehouse_name'],
                    'status': 'FAILED - Write Error',
                    'timestamp': datetime.now().isoformat()
                })
                print(f"\n✗ {file_config['lakehouse_name']} - WRITE FAILED")
        else:
            failed_count += 1
            results.append({
                'file': file_config['lakehouse_name'],
                'status': 'FAILED - Download Error',
                'timestamp': datetime.now().isoformat()
            })
            print(f"\n✗ {file_config['lakehouse_name']} - DOWNLOAD FAILED")
    
    # Print summary
    print("\n" + "=" * 80)
    print(f"ETL Process Complete - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("=" * 80)
    print(f"✓ Success: {success_count}")
    print(f"✗ Failed:  {failed_count}")
    print("\nDetailed Results:")
    for result in results:
        status_icon = "✓" if result['status'] == 'SUCCESS' else "✗"
        print(f"  {status_icon} {result['file']}: {result['status']}")
    print("=" * 80)
    
    return success_count, failed_count, results

## ▶️ Execute ETL

Run the cell below to start the ETL process.

**What happens:**
1. Downloads each file from SharePoint
2. Writes to Lakehouse (overwrites if exists)
3. Displays progress and results

---

In [ ]:
# Execute the ETL process
success, failed, results = run_etl()

## Verification

Run the cell below to verify files were successfully written to the Lakehouse.

---

In [ ]:
# Verify files were written to lakehouse
print("\nFiles in Lakehouse:")
print("=" * 80)
try:
    files = mssparkutils.fs.ls(lakehouse_abfs_path)
    for file in files:
        print(f"  📄 {file.name} ({file.size:,} bytes)")
except Exception as e:
    print(f"Could not list files: {str(e)}")
print("=" * 80)

## 🏁 Completion

The cell below exits the notebook with the appropriate status code.

**Exit Codes:**
- Success: All files processed successfully
- Partial: Some files failed
- ✗ Failure: All files failed

---

In [ ]:
# Exit notebook with appropriate status
if failed > 0:
    exit_msg = f"ETL completed with {failed} failures and {success} successes"
    print(f"\n⚠️  {exit_msg}")
    mssparkutils.notebook.exit(exit_msg)
else:
    exit_msg = f"ETL completed successfully. All {success} files processed."
    print(f"\n✓ {exit_msg}")
    mssparkutils.notebook.exit(exit_msg)

## 🔧 Troubleshooting

### Common Issues:

**❌ "No sharing link configured"**
- Ensure you've added the sharing link in Cell 6 configuration

**❌ "Download failed: Status 403"**
- Sharing link permissions may be restricted
- Verify "Anyone with link can edit" is enabled

**❌ "Download failed: Status 404"**
- File not found or sharing link expired
- Regenerate the sharing link

**❌ "Content is not an Excel file"**
- Verify the file is actually an Excel file (.xlsx)
- Check if the sharing link points to the correct file

**❌ "Error writing to lakehouse"**
- Verify lakehouse path is correct
- Check workspace permissions

---

## 📅 Scheduling

To run this notebook automatically:
1. Create a Fabric Pipeline
2. Add a Notebook activity
3. Select this notebook
4. Configure a Schedule trigger (daily, hourly, etc.)
5. Save and activate

---

## 📝 Notes

- Files are **overwritten** on each run
- Process continues even if individual files fail
- Check the summary for detailed status of each file